In [296]:
log_file = "/fsx/home-meng/hf_spaces/stablelm-jp-alpha/log.jsonl"
fbs = []

prefix = "INFO:root:"
with open(log_file, "r") as fp:
    for l in fp.readlines():
        if not l.startswith(prefix): continue
        
        l = l.replace(prefix, "")
        l = l[:-1] if l.endswith("\n") else l
        
        fbs.append(eval(l))
        # break

In [297]:
df = pd.DataFrame.from_records(fbs)
df = df[~df.duplicated(["prompt", "generation", "rating"])]
df.shape

(3255, 10)

In [298]:
df.rating.describe()

count     447
unique      5
top       不合格
freq      184
Name: rating, dtype: object

In [299]:
df.rating.value_counts(normalize=True)

不合格    0.411633
最悪     0.299776
中立     0.134228
合格     0.111857
最高     0.042506
Name: rating, dtype: float64

In [300]:
df.groupby("rating").temperature.describe()

,count,mean,std,min,25%,50%,75%,max
rating,,,,,,,,
不合格,184.0,0.681250,0.129581,0.05,0.7,0.7,0.7,1.00
中立,60.0,0.686667,0.081753,0.15,0.7,0.7,0.7,0.85
合格,50.0,0.621000,0.180444,0.05,0.7,0.7,0.7,0.85
最悪,134.0,0.682836,0.143021,0.00,0.7,0.7,0.7,1.00
最高,19.0,0.650000,0.160728,0.05,0.7,0.7,0.7,0.70


In [301]:
df.groupby("rating").max_new_tokens.describe()

,count,mean,std,min,25%,50%,75%,max
rating,,,,,,,,
不合格,184.0,177.391304,109.980918,128.0,128.0,128.0,128.0,512.0
中立,60.0,195.200000,142.098176,128.0,128.0,128.0,128.0,512.0
合格,50.0,166.400000,104.244610,128.0,128.0,128.0,128.0,512.0
最悪,134.0,167.164179,101.999351,128.0,128.0,128.0,128.0,512.0
最高,19.0,161.684211,93.887079,128.0,128.0,128.0,128.0,512.0


In [302]:
from IPython.display import display

In [303]:
# for r in df.rating.unique():
#     print(r)
#     display(df[df.rating == r].head(1))

In [304]:
df.columns

Index(['prompt', 'max_new_tokens', 'temperature', 'repetition_penalty',
       'do_sample', 'no_repeat_ngram_size', 'generation', 'time', 'rating',
       'version'],
      dtype='object')

In [ ]:
# df = df.loc[:, ['rating', 'prompt', 'generation', 'time', 'temperature', 'repetition_penalty']]


In [277]:
# df = df.sort_values("rating")

In [ ]:
n = len(df)

In [305]:
df['prompt'] = df.prompt.apply(lambda x: x.strip())

In [306]:
question = df.prompt.str.endswith("？") \
    | df.prompt.str.endswith("教えて。") \
    | df.prompt.str.endswith("教えて") \
    | df.prompt.str.endswith("ね")

command = df.prompt.str.endswith("ださい") \
    | df.prompt.str.endswith("！") \

say_hi = df.prompt.str.contains("こんにちは") \
    | df.prompt.str.endswith("よろしく")

natural_cont = df.prompt.str.endswith("は") \
    | df.prompt.str.endswith("は、") \
    | df.prompt.str.endswith("、") \
    | df.prompt.str.endswith("「")

# english = df.prompt.apply(lambda x: True if x.isalnum() else False)

categorized = question | command | say_hi | natural_cont

i = 0
for t in df[~categorized].itertuples():
    i += 1
    # print(i, t.prompt[:100])
    # print()

In [307]:
types =      [question, command, say_hi, natural_cont]
type_names = ['question', 'command', 'say_hi', 'natural_cont']
for t, name in zip(types, type_names):
    d = df[t]
    print(d.shape[0], name, d.shape[0] / n)
    df.loc[t, 'type'] = name
    # print(display(d.head(2)))

239 question 0.08380084151472651
98 command 0.034361851332398316
201 say_hi 0.07047685834502104
1239 natural_cont 0.4344319775596073


In [308]:
df.loc[df['type'].isna(), ['type']] = 'others'

In [309]:
df['type'].value_counts(normalize=True)

others          0.500768
natural_cont    0.380645
question        0.072197
command         0.028879
say_hi          0.017512
Name: type, dtype: float64

In [319]:
df['type'].value_counts(normalize=True)

others          0.500768
natural_cont    0.380645
question        0.072197
command         0.028879
say_hi          0.017512
Name: type, dtype: float64

In [314]:
df.groupby(['type']).rating.value_counts(ascending=False, normalize=True).to_frame().round(2)

rating
type         rating        
command      最悪        0.46
             不合格       0.38
             中立        0.08
             最高        0.08
natural_cont 不合格       0.39
             最悪        0.27
             中立        0.16
             合格        0.15
             最高        0.03
others       不合格       0.43
             最悪        0.28
             中立        0.14
             合格        0.09
             最高        0.06
question     最悪        0.53
             不合格       0.42
             合格        0.03
             最高        0.03
say_hi       不合格       0.71
             合格        0.14
             最悪        0.14

In [316]:
df = df.loc[:, ['type', 'rating', 'prompt', 'generation', 'version', 'time']]

In [317]:
df.to_csv("/fsx/home-meng/hf_spaces/stablelm-jp-alpha/processed_app_log.tsv", sep="\t")